In [1]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 14.1 MB/s eta 0:00:00


In [2]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [3]:
!pip install pyngrok


In [41]:
%%writefile app.py


import streamlit as st
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import io
import sqlite3

st.header("Business Card Data Extraction", divider="rainbow")
st.title("Image Upload App")

# aa = None
# ab = None
# ac = None
# ad = None
# ae = None
# af = None
# ag = None
# ah = None
# ai = None
# aj = None

import streamlit as st
def Ex_data(uploaded_image):
  image = Image.open(uploaded_image)
  aad = st.image(image, caption='Uploaded Image', use_column_width=True)

  reader = easyocr.Reader(['en'])
  image = uploaded_image.name
  a = reader.readtext(image, detail=0)
  data= {'Name': [], 'Designation': [], 'Company_name': [], 'Contact': [], 'Email': [], 'Website': [],
                        'Area': [], 'District' :[], 'State':[], 'Pincode': []}

  for i in range(2, len(a)):
    if "-" in a[i]:
      d = a[i].split(", ")[0]
      data["Contact"].append(d)
      break

  for i in range(2, len(a)):
    if '@' in a[i] and '.com' in a[i]:
      data['Email'].append(a[i])

    if 'www' in a[i] or 'WWW' in a[i] or 'wWW' in a[i] and 'com' in a[i]:
      data['Website'].append(a[i])

    match1 = re.findall('.+St , ([a-zA-Z]+).+', a[i])
    match2 = re.findall('.+St,, ([a-zA-Z]+).+', a[i])
    match3 = re.findall('^[E].*', a[i])

    if match1:
      data["District"].append(match1[0])
    elif match2:
      data["District"].append(match2[0])
    elif match3:
      data["District"].append(match3[0])

    if 'St' in a[i] :
      e = a[i].split(",")[0].strip()
      data['Area'].append(e)

    if 'TamilNadu' in a[i] or 'Tamil Nadu' in a[i]:
      b = 'TamilNadu'
      data["State"].append(b)

    if 'TamilNadu' in a[i] or 'Tamil Nadu' in a[i] or a[i].isdigit()>=6:
      c = a[i].split(" ")[1]
      if c.isdigit() == True:
        data['Pincode'].append(c)
    elif a[i].isdigit() and len(a[i]) >=6:
      data['Pincode'].append(a[i])


  data['Name'].append(a[0])
  a.pop(0)
  data['Designation'].append(a[0])
  a.pop(0)

  i = 0
  while i < len(a):
    if re.match(r'^[A-Za-z ]+$', a[i]):
      data['Company_name'].append(a[i])
      a.pop(i)
    else:
      i += 1

  company_name = ' '.join(data['Company_name']).replace(',', '')
  data['Company_name'] = [company_name]
  # data2.append(data.copy())
  ext_data = data
  data1 = pd.DataFrame(data)
  # st.write(ext_data)

  conn = sqlite3.connect('my_database25.db')
  cursor = conn.cursor()

  cursor.execute('''CREATE TABLE IF NOT EXISTS Contacts (
                  Name TEXT,
                  Designation TEXT,
                  Company_name TEXT,
                  Contact TEXT,
                  Email TEXT,
                  Website TEXT,
                  Area TEXT,
                  District TEXT,
                  State TEXT,
                  Pincode TEXT)''')

  cursor.execute('''INSERT INTO Contacts (Name, Designation, Company_name, Contact, Email, Website, Area, District, State, Pincode)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                      (ext_data['Name'][0], ext_data['Designation'][0], ext_data['Company_name'][0], ext_data['Contact'][0], ext_data['Email'][0],
                      ext_data['Website'][0], ext_data['Area'][0], ext_data['District'][0], ext_data['State'][0], ext_data['Pincode'][0]))
  conn.commit()
  conn.close()

uploaded_image = st.file_uploader("Upload Image", type=["jpg", "png", "jpeg"])
if uploaded_image is not None and st.button('upload'):
  Ex_data(uploaded_image)


col1,col2 = st.columns(2,gap="medium")
with col1:
  conn = sqlite3.connect('my_database25.db')
  cursor = conn.cursor()
  cursor.execute('select * from Contacts')
  rows = cursor.fetchall()
  aa=st.text_input('Name',rows[-1][0])
  ab=st.text_input('Designation',rows[-1][1])
  ac=st.text_input('Company_name',rows[-1][2])
  ad=st.text_input('Contact', rows[-1][3])
  ae=st.text_input('Email', rows[-1][4])
  af=st.text_input('Website', rows[-1][5])
  ag=st.text_input('Area', rows[-1][6])
  ah=st.text_input('District', rows[-1][7])
  ai=st.text_input('State', rows[-1][8])
  aj=st.text_input('Pincode', rows[-1][9])

if st.button('update'):
  # st.write(aa)s
  conn = sqlite3.connect('my_database25.db')
  cursor = conn.cursor()

  q1=f"UPDATE Contacts SET Name = '{aa}', Designation = '{ab}', Company_name = '{ac}', Contact = '{ad}', Email = '{ae}', Website = '{af}', Area = '{ag}', District = '{ah}', State = '{ai}',pincode = '{aj}' WHERE pincode = '{aj}';"
  # st.write(q1)
  cursor.execute(q1)
  conn.commit()
  cursor.execute('select * from Contacts')
  rows2 = cursor.fetchall()
  st.table(rows2)

with col2:
  conn = sqlite3.connect('my_database25.db')
  cursor = conn.cursor()
  cursor.execute("select name from Contacts")
  rows3 = cursor.fetchall()
  names = ["Select"]
  for i in rows3:
    names.append(i[0])
  st.selectbox("Select the name to delete", options=names)
  if st.button("Clik here to delete"):
    cursor.execute(f"DELETE FROM Contacts WHERE NAME = '{aa}'")
    conn.commit()
    cursor.execute('select * from Contacts')
    rows4 = cursor.fetchall()
    st.table(rows4)


Overwriting app.py


In [42]:

!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.656s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.23.102.94
npx: installed 22 in 2.754s
your url is: https://fresh-buses-appear.loca.lt
